# Reinforcement learning bots

In [ ]:
import sys
import os
import numpy as np

project_root = os.path.abspath("../../..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt

%config InlineBackend.figure_formats = ['svg']
plt.style.use("ggplot")


## Crafting the environment

- How to represent observation space?
  

In [5]:
rng = np.random.default_rng(2137)

In [22]:
from typing import Optional
import gymnasium as gym
import numpy as np

from catan.core.game import Game as CatanGame
from catan.core.models.enums import Action as CatanAction, ActionType
from catan.core.models.map import DEFAULT_MAP
from catan.bots.rl.rewards import action_to_reward
from catan.core.models.player import Player, RandomPlayer


N_TILES = len(DEFAULT_MAP.land_tiles)
N_NODES = len(DEFAULT_MAP.land_nodes)
N_EDGES = 72

# settlement/city/none
# road or no road
# tile (resource, robber, num)
OBS_SPACE = (
    (N_NODES * 3, "NODES"),
    (N_EDGES * 2, "EDGES"),
    (N_TILES * 3, "TILES"),
)

ACTION_SPACE = (
    *N_NODES * (ActionType.BUILD_SETTLEMENT,),
    *N_NODES * (ActionType.BUILD_CITY,),
    *N_EDGES * (ActionType.BUILD_ROAD,),
    *N_TILES * (ActionType.MOVE_ROBBER,),
    *1 * (ActionType.PLAY_KNIGHT_CARD,),
    *1 * (ActionType.PLAY_ROAD_BUILDING,),
    *9 * (ActionType.MARITIME_TRADE,),
    *1 * (ActionType.ROLL,),
    *1 * (ActionType.BUY_DEVELOPMENT_CARD,),
    *1 * (ActionType.DISCARD,),
    *1 * (ActionType.END_TURN,),
)


class RLPlayer(Player):
    def decide(self, game, playable_actions):
        pass


class CatanEnv(gym.Env):
    def __init__(self) -> None:
        self.render_mode = None
        self.np_random = rng

        self.observation_space = gym.spaces.Box(
            low=0,
            high=max(OBS_SPACE, key=lambda k: k[0])[0],
            shape=(sum(map(lambda k: k[0], OBS_SPACE)),),
            dtype=np.int8,
        )
        self.action_space = gym.spaces.Discrete(len(ACTION_SPACE))

        self._player_location = self.observation_space
        self._game = CatanGame(players=[RandomPlayer])

    def _action_to_catan_action(env_action: np.int8) -> CatanAction:
        action_type = ACTION_SPACE[env_action]

    def _random_agent_state():
        pass

    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)

    def _get_info(self):
        pass

    def _is_terminated(self):
        return self._game.winning_color() is not None

    def step(self, action: np.int8):
        catan_action = self._action_to_catan_action(action)
        self._game.execute(action)

        next_obs = self._get_obs()
        reward = action_to_reward(catan_action)
        terminated = self._is_terminated()
        truncated = False
        info = self._get_info()

        return next_obs, reward, terminated, truncated, info

In [23]:
ACTION_SPACE

(<ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType.BUILD_SETTLEMENT: 'BUILD_SETTLEMENT'>,
 <ActionType